In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train = pd.read_csv(r"C:\Users\RISHI MUKUNTHAN\Desktop\Data Science\Projects\Edwisor_Santander_Customer_Transaction\train.csv")
df_test = pd.read_csv(r"C:\Users\RISHI MUKUNTHAN\Desktop\Data Science\Projects\Edwisor_Santander_Customer_Transaction\test.csv")

#### UnderSampling the majority class

In [3]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df_train[df_train.target==0]
df_minority = df_train[df_train.target==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=150000,
                                 random_state=42) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [4]:
X = df_downsampled.iloc[:, 2:].values
y = df_downsampled.iloc[:, 1].values

In [5]:
from imblearn.over_sampling import RandomOverSampler

In [6]:
ros = RandomOverSampler(sampling_strategy={1:25000})
X_res, y_res = ros.fit_sample(X, y)

In [7]:
#Under 60 minutes Original with 200 variables
# Training XGBoost on the Training set
from xgboost import XGBClassifier
XGB_classifier = XGBClassifier(n_estimators=200, subsample=1, learning_rate=0.2, gamma=1,
                               max_depth=20, min_child_weight=0.5, reg_alpha=0, reg_lambda=100,
                               colsample_bylevel=0.9, colsample_bytree=1, scale_pos_weight=6)
XGB_classifier.fit(X_res, y_res,eval_metric="auc", verbose=2)

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
              colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=20,
              min_child_weight=0.5, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=100, scale_pos_weight=6, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
import joblib
joblib.dump(XGB_classifier,'XGB_Model_Final.pkl')

['XGB_Model_Final.pkl']

In [15]:
model = joblib.load('XGB_Model_Final.pkl')

In [16]:
X_test = df_test.iloc[:, 1:].values

In [17]:
y_score = model.predict_proba(X_test)
y_pred = (y_score[:,1] >= 0.5).astype(int)
df_test_submit = df_test['ID_code']
df_test_submit = pd.concat([df_test_submit,pd.Series(y_pred),pd.Series(y_score[:,1])], axis=1)
df_test_submit.columns =['ID_code','Target','Probability']
df_test_submit.to_csv('Test_Prediction_XGB.csv', index=False)

In [18]:
df_check = pd.read_csv(r"C:\Users\RISHI MUKUNTHAN\Desktop\Data Science\Projects\Edwisor_Santander_Customer_Transaction\Python Development\Test_Prediction_XGB.csv")

In [19]:
df_check.head()

,ID_code,Target,Probability
0,test_0,0,0.098175
1,test_1,0,0.301806
2,test_2,0,0.077992
3,test_3,0,0.073081
4,test_4,0,0.014669
